In [1]:
import glob
import pandas as pd
import json
import datetime
from tqdm import tqdm
import os
from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm, tqdm_notebook
import glob
import numpy as np

from django.db.models import Min,Max,Count
from sklearn.neighbors import KDTree
comfig_file_path = '../../main_config.txt'
main_name = 'yandex/'
sub_name = 'raw_org_search/'
import time
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from itertools import product
import numpy as np
from sklearn.utils import shuffle

/home/tserekh/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/tserekh/env/lib/python3.6/site-packages/django/db/models/base.py:299: RuntimeWarning: Model 'main_map.predictedsquare' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


In [2]:
import sys
import json
sys.path.append('/usr/lib/python3/dist-packages')
from main_map.models import OrganizationNatClass, House, Metro, DistanceTemp, City

In [3]:
#nat_classes = ['АЗС','Парикмахерская', 'Ресторан','Супермаркет']
true_nat_classes_list = [
#     ['Супермаркет','Магазин продуктов'],
#     ['Парикмахерская'],
#     ['АЗС'],
    ['Быстрое питание'],
]
#nat_classes = ['АЗС']
#nat_classes = ['Автомойка']

In [4]:
def calc_distance(df1, df2, from_2, k):
    
    tree = KDTree(df1[xy])
    distances_list, indexes_list = tree.query(df2[xy], k=k)
    
    df1 = df1.rename(columns = {'source_id':'source_id1','source_name':'source_name1'})
    df2 = df2.rename(columns = {'source_id':'source_id2','source_name':'source_name2'})

    df1_all = pd.DataFrame()
    df_orgs_all = pd.DataFrame()
    for i in tqdm_notebook(range(k), leave=False):
        indexes = indexes_list[:,i]
        distances = distances_list[:,i]
        df1_sub = df1.iloc[indexes].copy()

        for col in from_2:

            df1_sub[col] = df2[col].values
        df1_sub['distance'] = distances
        df1_all = df1_all.append(df1_sub)
    return df1_all

def get_result(df_three, score_name):
    df_gb1 = df_three[df_three['true']].groupby(['source_id1','source_name1'],as_index=False)[score_name].sum()
    df_gb2 = df_three[df_three['true']==False].groupby(['source_id1','source_name1'],as_index=False)[score_name].sum()
    df_gb = pd.concat([df_gb1, df_gb2])

    df = df_three.copy()
    df = pd.merge(df, df_gb,on=['source_id1','source_name1'])
    df['rate'] = df[score_name + '_x']/(df[score_name +'_y'] + df[score_name +'_y']*(df['true']==False))
    df['our_flats'] = df['flat_num']*df['rate'] 

    agg = {}
    agg['our_flats'] = sum
    agg['true'] = lambda x: list(x)[0]
    df_result = df.groupby(['source_id1','source_name1'],as_index=False).agg(agg)
    return df_result

In [5]:
lat_min, lon_min = 55.561417, 37.245485
lat_max, lon_max = 55.933221, 37.875825




filter_geo =    {
    'lat__gt' : lat_min, 
    'lon__gt' : lon_min,
    'lat__lt' : lat_max, 
    'lon__lt' : lon_max,
}
filter_dic = filter_geo.copy()

In [27]:
def f(df_three, x1,x2,w):
    df_three['extra_distance'] = df_three['direct_distance'] - df_three['first_distance'] - df_three['second_distance']

    df_three['two_score'] = (df_three['second_distance']+1)**x1
    df_three['three_score'] = (df_three['extra_distance']+1)**x2


    
    
    df_result_two = get_result(df_three, 'two_score')
    df_result_three = get_result(df_three, 'three_score')
    pred = df_result_two['our_flats']*w + (1 - w)*df_result_three['our_flats']
    fact = df_result_two['true'].values
    auc_roc = roc_auc_score(fact, pred)
    return auc_roc

In [20]:
x1_list  = [1]
x2_list = np.log(np.linspace(10**(-0.2),10**(-5),20))
x3_list = x2_list.copy()
w_list =  np.linspace(0,1,6)

In [21]:
x2_list

array([ -0.46051702,  -0.51458336,  -0.57174079,  -0.6323643 ,
        -0.69690157,  -0.76589301,  -0.83999933,  -0.9200401 ,
        -1.0070492 ,  -1.10235664,  -1.20771381,  -1.32549266,
        -1.45901868,  -1.61316219,  -1.79547371,  -2.0186022 ,
        -2.30625919,  -2.71167411,  -3.40467076, -11.51292546])

In [22]:
combs = product(x1_list, x2_list,x3_list, w_list)
combs = list(combs)
combs = shuffle(combs)

In [23]:
columns = ['address', 'flat_num', 'source_id', 'source_name',  'x', 'y']
df_homes = pd.DataFrame(list(House.objects.filter(**filter_geo).values().values_list(*columns)),columns=columns)

In [26]:
from bayes_opt import BayesianOptimization

In [28]:
for true_nat_classes in tqdm_notebook(true_nat_classes_list):
    print(true_nat_classes)
    columns = ['address', 'chain_name', 'nat_class', 'source_id','source_name', 'x', 'y']

    filter_dic['nat_class__in'] = true_nat_classes
    df_orgs1 = pd.DataFrame(list(OrganizationNatClass.objects.filter(**filter_dic).values().values_list(*columns)),columns=columns)
    df_orgs1 = df_orgs1.drop_duplicates()
    df_orgs1['true'] = True

    df_orgs2 = pd.DataFrame(
        list(OrganizationNatClass
             .objects.filter(**filter_geo)
             .exclude(**{'source_id__in':df_orgs1['source_id'].values})
             .values()
             .values_list(*columns).order_by('?')[:10000]),columns=columns)
    df_orgs2 = df_orgs2.drop_duplicates()
    df_orgs2['true'] = False
    df_orgs = pd.concat([df_orgs1, df_orgs2])

    xy = ['x','y']
    columns = ['source_id','source_name','x','y']
    df_cities = pd.DataFrame(list(Metro.objects.filter(**filter_geo).values().values_list(*columns)),columns=columns)

    df_cities_homes = calc_distance(df_cities[columns], df_homes[columns],['source_id2','source_name2'], 10)
    df_cities_homes = df_cities_homes.rename(columns={
        'source_id2':'source_id3',
        'source_name2':'source_name3',
        'distance':'direct_distance',
    })

    df_cities_orgs = calc_distance(df_cities[columns], df_orgs[columns],['source_id2','source_name2'], 10)
    df_cities_orgs = df_cities_orgs.rename(columns={
        'distance':'first_distance',

    })

    df_orgs_homes = calc_distance(df_orgs[columns], df_homes[columns],['source_id2','source_name2'], 20)
    df_orgs_homes = df_orgs_homes.rename(columns={
        'source_id2':'source_id3',
        'source_name2':'source_name3',
        'source_id1':'source_id2',
        'source_name1':'source_name2',
        'distance':'second_distance',

    })

    df12 = pd.merge(
        df_cities_orgs[['source_id1','source_name1','source_id2','source_name2','first_distance']],
        df_orgs_homes[['source_id2','source_name2','source_id3','source_name3','second_distance']],
        on=['source_id2','source_name2']
    )

    df_three = pd.merge(df12, df_cities_homes,on=['source_id1','source_name1','source_id3','source_name3'])
    df_three = df_three.drop(['source_id1','source_name1'],axis=1)
    df_three = df_three.drop_duplicates()

    df_three = df_three.rename(columns={
        'source_id3':'source_id2',
        'source_name3':'source_name2',
        'source_id2':'source_id1',
        'source_name2':'source_name1',

    })
    df_three = pd.merge(
        df_orgs[['source_id','source_name','true']] .rename(
            columns = {'source_id':'source_id1','source_name':'source_name1'}),
        df_three,
        on=['source_id1','source_name1']
    )

    df_three = pd.merge(
        df_homes[['source_id','source_name','flat_num']] .rename(
            columns = {'source_id':'source_id2','source_name':'source_name2'}),
        df_three,
        on=['source_id2','source_name2']
    )
    best_auc_roc = 0
    best_params = None

    ind = 0
    break

['Быстрое питание']


In [48]:
dic = {}
dic['x1'] = (-10,0)
dic['x2'] = (-10,0)
dic['w'] = (0.01,0.99)

In [49]:
def f_(x1,x2,w):
    return f(df_three,x1,x2,w)

In [50]:
opt = BayesianOptimization(f_,dic)

In [51]:
opt.maximize(n_iter=1000)

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |         w |        x1 |        x2 | 
    1 | 00m19s |    0.67340 |    0.4293 |   -8.5684 |   -3.3444 | 
    2 | 00m20s |    0.68837 |    0.5580 |   -4.6836 |   -9.5048 | 
    3 | 00m19s |    0.68716 |    0.8080 |   -9.5379 |   -3.8799 | 
    4 | 00m16s |    0.69660 |    0.7425 |   -4.7235 |   -5.5056 | 
    5 | 00m15s |    0.65921 |    0.3259 |   -9.4836 |   -7.5170 | 
Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |         w |        x1 |        x2 | 
    6 | 00m23s |    0.77322 |    0.9900 |    0.0000 |    0.0000 | 
    7 | 00m23s |    0.77322 |    0.9900 |    0.0000 |  -10.0000 | 
    8 | 00m20s |    0.77304 |    0.9900 |    0.0000 |   -4.7002 | 
    9 | 00m23s |    0.77304 |    0.9900 |   -0.0000 |   -6.9514 | 


/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   10 | 00m31s |    0.77303 |    0.9900 |    0.0000 |   -1.8104 | 
   11 | 00m32s |    0.77304 |    0.9900 |    0.0000 |   -8.0558 | 


/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.09151631e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00031058]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   12 | 00m30s |    0.77265 |    0.9859 |   -0.0252 |   -8.7333 | 


...

/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00032611]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


  118 | 00m49s |    0.68622 |    0.9900 |   -6.8009 |    0.0000 | 


/home/tserekh/env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-5.4840938e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


  119 | 00m46s |    0.68763 |    0.9900 |   -6.1183 |   -4.9731 | 
  120 | 00m49s |    0.68028 |    0.3291 |    0.0000 |   -3.6365 | 
  121 | 00m49s |    0.59166 |    0.0100 |   -1.5798 |   -8.9109 | 
  122 | 00m50s |    0.68789 |    0.9900 |   -6.0356 |   -7.3702 | 
  123 | 00m52s |    0.73857 |    0.4340 |   -9.4168 |    0.0000 | 
  124 | 00m50s |    0.71945 |    0.9900 |   -2.1106 |   -4.6670 | 


KeyboardInterrupt: 

In [25]:
    for comb in tqdm_notebook(combs, leave=False):
        auc_roc = f(df_three,*comb)
        print(ind,')', 'auc_roc',auc_roc,comb)
        if auc_roc>best_auc_roc:
            best_auc_roc = auc_roc
            best_params = comb
            print('BEST',ind,')', 'auc_roc',best_auc_roc,best_params)
        ind+=1

['Быстрое питание']


0 ) auc_roc 0.6527883960193357 (1, -1.6131621897742732, -0.632364303855148, 0.2)
BEST 0 ) auc_roc 0.6527883960193357 (1, -1.6131621897742732, -0.632364303855148, 0.2)
1 ) auc_roc 0.7030302355240027 (1, -0.8399993254388091, -0.632364303855148, 0.4)
BEST 1 ) auc_roc 0.7030302355240027 (1, -0.8399993254388091, -0.632364303855148, 0.4)
2 ) auc_roc 0.6957689233547603 (1, -1.4590186794814382, -1.6131621897742732, 0.4)
3 ) auc_roc 0.5895896212387075 (1, -0.9200401028096967, -0.9200401028096967, 0.0)
4 ) auc_roc 0.6865481605975612 (1, -2.7116741103573196, -3.404670757675428, 0.4)
5 ) auc_roc 0.7476468961085938 (1, -0.9200401028096967, -1.0070491985373635, 0.8)
BEST 5 ) auc_roc 0.7476468961085938 (1, -0.9200401028096967, -1.0070491985373635, 0.8)
6 ) auc_roc 0.5807225612234239 (1, -0.632364303855148, -2.7116741103573196, 0.0)
7 ) auc_roc 0.5852081127061699 (1, -3.404670757675428, -1.6131621897742732, 0.0)
8 ) auc_roc 0.5861240869269609 (1, -1.6131621897742732, -1.4590186794814382, 0.0)
9 ) auc_

93 ) auc_roc 0.7291715220875907 (1, -1.7954737093063908, -2.7116741103573196, 1.0)
94 ) auc_roc 0.7330262060033099 (1, -1.6131621897742732, -0.5145833593732545, 1.0)
95 ) auc_roc 0.7334736289831392 (1, -1.6131621897742732, -1.0070491985373635, 0.8)
96 ) auc_roc 0.764974168078464 (1, -0.5145833593732545, -0.632364303855148, 1.0)
97 ) auc_roc 0.7320579855386792 (1, -0.6969015702901241, -11.512925464970229, 0.6000000000000001)
98 ) auc_roc 0.6508081834600378 (1, -1.0070491985373635, -1.7954737093063908, 0.2)
99 ) auc_roc 0.6491917299623751 (1, -0.8399993254388091, -3.404670757675428, 0.2)
100 ) auc_roc 0.7602066067556423 (1, -0.632364303855148, -2.7116741103573196, 1.0)
101 ) auc_roc 0.6525153269800702 (1, -1.3254926640414657, -0.9200401028096967, 0.2)
102 ) auc_roc 0.7319428872965938 (1, -0.8399993254388091, -2.0186022049167422, 0.6000000000000001)
103 ) auc_roc 0.6548284137117987 (1, -0.6969015702901241, -0.632364303855148, 0.2)
104 ) auc_roc 0.6495185104046447 (1, -0.9200401028096967, 

189 ) auc_roc 0.6953543832084501 (1, -0.9200401028096967, -11.512925464970229, 0.4)
190 ) auc_roc 0.7333213190281852 (1, -0.7658930078331804, -2.0186022049167422, 0.6000000000000001)
191 ) auc_roc 0.7264054847300945 (1, -1.3254926640414657, -1.0070491985373635, 0.6000000000000001)
192 ) auc_roc 0.5890472126562314 (1, -0.5717407891305454, -1.0070491985373635, 0.0)
193 ) auc_roc 0.755611184985102 (1, -0.632364303855148, -0.8399993254388091, 0.8)
194 ) auc_roc 0.6521066141713459 (1, -11.512925464970229, -0.4605170185988091, 0.2)
195 ) auc_roc 0.7575807085423141 (1, -0.5717407891305454, -0.6969015702901241, 0.8)
196 ) auc_roc 0.7333016059776095 (1, -1.6131621897742732, -1.2077138106597225, 0.8)
197 ) auc_roc 0.6850292067824749 (1, -3.404670757675428, -2.7116741103573196, 0.4)
198 ) auc_roc 0.6995383283472968 (1, -0.7658930078331804, -2.7116741103573196, 0.4)
199 ) auc_roc 0.6921817974819042 (1, -2.7116741103573196, -0.5145833593732545, 0.4)
200 ) auc_roc 0.7365415890257181 (1, -1.459018679

285 ) auc_roc 0.7418187759947124 (1, -0.4605170185988091, -0.632364303855148, 0.6000000000000001)
286 ) auc_roc 0.7063661698478435 (1, -3.404670757675428, -2.0186022049167422, 1.0)
287 ) auc_roc 0.5819880758021405 (1, -1.0070491985373635, -2.3062591850324683, 0.0)
288 ) auc_roc 0.7311989194317967 (1, -0.8399993254388091, -2.7116741103573196, 0.6000000000000001)
289 ) auc_roc 0.5916417764428846 (1, -0.9200401028096967, -0.4605170185988091, 0.0)
290 ) auc_roc 0.7248623224897236 (1, -2.0186022049167422, -0.5145833593732545, 1.0)
291 ) auc_roc 0.6817513793391313 (1, -11.512925464970229, -0.5717407891305454, 1.0)
292 ) auc_roc 0.7504670777570291 (1, -0.7658930078331804, -3.404670757675428, 0.8)
293 ) auc_roc 0.7411159990818781 (1, -0.4605170185988091, -1.102356640834202, 0.6000000000000001)
294 ) auc_roc 0.6533697811654922 (1, -1.3254926640414657, -0.5717407891305454, 0.2)
295 ) auc_roc 0.7413716194074583 (1, -1.2077138106597225, -1.0070491985373635, 0.8)
296 ) auc_roc 0.7408686868747414 (1

...

475 ) auc_roc 0.6533269585590054 (1, -1.2077138106597225, -0.6969015702901241, 0.2)
476 ) auc_roc 0.5831480489724123 (1, -0.9200401028096967, -2.0186022049167422, 0.0)
477 ) auc_roc 0.7477129614672796 (1, -0.9200401028096967, -0.9200401028096967, 0.8)
478 ) auc_roc 0.7426430612209765 (1, -1.2077138106597225, -1.3254926640414657, 1.0)
479 ) auc_roc 0.764974168078464 (1, -0.5145833593732545, -1.0070491985373635, 1.0)
480 ) auc_roc 0.7503278876364109 (1, -0.9200401028096967, -0.9200401028096967, 1.0)
481 ) auc_roc 0.7537570094130149 (1, -0.6969015702901241, -0.6969015702901241, 0.8)
482 ) auc_roc 0.6550942568524578 (1, -0.4605170185988091, -0.9200401028096967, 0.2)
483 ) auc_roc 0.7042994296501662 (1, -0.5717407891305454, -1.3254926640414657, 0.4)
484 ) auc_roc 0.737463623677598 (1, -0.5145833593732545, -3.404670757675428, 0.6000000000000001)
485 ) auc_roc 0.5915028360653057 (1, -0.4605170185988091, -0.5145833593732545, 0.0)
486 ) auc_roc 0.709639968103485 (1, -2.7116741103573196, -2.0186

571 ) auc_roc 0.6557000335687944 (1, -0.5145833593732545, -0.5145833593732545, 0.2)
572 ) auc_roc 0.652594029336549 (1, -0.632364303855148, -1.6131621897742732, 0.2)
573 ) auc_roc 0.5842145016992016 (1, -0.5145833593732545, -1.7954737093063908, 0.0)
574 ) auc_roc 0.7162116231942288 (1, -2.0186022049167422, -1.7954737093063908, 0.6000000000000001)
575 ) auc_roc 0.6495044082476873 (1, -2.0186022049167422, -1.6131621897742732, 0.2)
576 ) auc_roc 0.5890472126562314 (1, -2.7116741103573196, -1.0070491985373635, 0.0)
577 ) auc_roc 0.7045466253105512 (1, -3.404670757675428, -2.7116741103573196, 0.6000000000000001)
578 ) auc_roc 0.7464770997121063 (1, -0.9200401028096967, -3.404670757675428, 0.8)
579 ) auc_roc 0.7626593164959241 (1, -0.5717407891305454, -2.0186022049167422, 1.0)
580 ) auc_roc 0.7058008848029595 (1, -0.4605170185988091, -1.2077138106597225, 0.4)
581 ) auc_roc 0.5852081127061699 (1, -0.8399993254388091, -1.6131621897742732, 0.0)
582 ) auc_roc 0.7547208344001524 (1, -0.6323643038

667 ) auc_roc 0.7295969010818102 (1, -1.7954737093063908, -1.7954737093063908, 0.8)
668 ) auc_roc 0.6525678396164852 (1, -1.7954737093063908, -0.632364303855148, 0.2)
669 ) auc_roc 0.5869578124401136 (1, -0.9200401028096967, -1.3254926640414657, 0.0)
670 ) auc_roc 0.5907876885472536 (1, -0.5145833593732545, -0.6969015702901241, 0.0)
671 ) auc_roc 0.5831480489724123 (1, -1.2077138106597225, -2.0186022049167422, 0.0)
672 ) auc_roc 0.7606276235714406 (1, -0.4605170185988091, -1.3254926640414657, 0.8)
673 ) auc_roc 0.7602066067556423 (1, -0.632364303855148, -1.0070491985373635, 1.0)
674 ) auc_roc 0.7428131695298708 (1, -1.102356640834202, -2.0186022049167422, 0.8)
675 ) auc_roc 0.5842145016992016 (1, -1.102356640834202, -1.7954737093063908, 0.0)
676 ) auc_roc 0.7017375822016728 (1, -0.6969015702901241, -1.7954737093063908, 0.4)
677 ) auc_roc 0.7226392597989263 (1, -2.3062591850324683, -1.0070491985373635, 0.8)
678 ) auc_roc 0.739738233331725 (1, -1.3254926640414657, -11.512925464970229, 1.

764 ) auc_roc 0.7025069572583436 (1, -0.9200401028096967, -0.5145833593732545, 0.4)
765 ) auc_roc 0.5819880758021405 (1, -3.404670757675428, -2.3062591850324683, 0.0)
766 ) auc_roc 0.5869578124401136 (1, -0.5145833593732545, -1.3254926640414657, 0.0)
767 ) auc_roc 0.7552434633339091 (1, -0.7658930078331804, -0.6969015702901241, 1.0)
768 ) auc_roc 0.7024218781295924 (1, -0.9200401028096967, -0.5717407891305454, 0.4)
769 ) auc_roc 0.649281387713515 (1, -1.7954737093063908, -1.7954737093063908, 0.2)
770 ) auc_roc 0.5895896212387075 (1, -0.4605170185988091, -0.9200401028096967, 0.0)
771 ) auc_roc 0.690787831733263 (1, -11.512925464970229, -1.4590186794814382, 0.8)
772 ) auc_roc 0.7610783931079979 (1, -0.4605170185988091, -0.4605170185988091, 0.8)
773 ) auc_roc 0.7503278876364109 (1, -0.9200401028096967, -1.3254926640414657, 1.0)
774 ) auc_roc 0.7157824314562746 (1, -2.3062591850324683, -0.5717407891305454, 0.6000000000000001)
775 ) auc_roc 0.7049619479850515 (1, -0.5145833593732545, -1.325

KeyboardInterrupt: 